In [77]:
%%pyspark
#df = spark.read.load.('abfss://bronze@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer.txt', format='text', header='true')
df = spark.read.option("header",True).csv('abfss://bronze@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer.txt')

display(df.limit(10))

StatementMeta(testhmacspark, 23, 77, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0de35eb7-677f-4189-870b-3503f9b29567)

In [78]:
import hashlib
def fun_lst_10_hash():

    secret= TokenLibrary.getSecret('test-hmacmask-akv', 'username', 'AzureKeyVaultHMAC')
    lst_10 = list(range(0,10))
    lst_10_hash =[]
    for item in lst_10:
        value = str(item) + secret
        encoded_value = str.encode(value)
        hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()

        lst_10_hash.append(str(hash_obj_sha256))
        lst_10_hash.sort()
    return lst_10_hash

def fun_lst_100_hash():

    secret= TokenLibrary.getSecret('test-hmacmask-akv', 'username', 'AzureKeyVaultHMAC')
    lst_100 = list(range(0,100))
    lst_100_hash =[]
    for item in lst_100:

        if item < 10:
            value = "0" + str(item) +  secret
        else:
            value = str(item) + secret
        encoded_value = str.encode(value)
        hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()

        lst_100_hash.append(str(hash_obj_sha256))
        lst_100_hash.sort()
    return lst_100_hash
#

def fun_lst_1000_hash():

    secret= TokenLibrary.getSecret('test-hmacmask-akv', 'username', 'AzureKeyVaultHMAC')
    lst_1000 = list(range(0,1000))
    lst_1000_hash =[]
    for item in lst_1000:

        if item < 10:
            value = "00" + str(item) + secret
        elif item < 100:
            value = "0" + str(item) +  secret   
        else:
            value = str(item) + secret

        encoded_value = str.encode(value)
        hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()

        lst_1000_hash.append(str(hash_obj_sha256))
        lst_1000_hash.sort()
    return lst_1000_hash

def fun_lst_10000_hash():

    secret= TokenLibrary.getSecret('test-hmacmask-akv', 'username', 'AzureKeyVaultHMAC')
    lst_10000 = list(range(0,10000))
    lst_10000_hash =[]
    for item in lst_10000:

        if item < 10:
            value = "000" + str(item) + secret
        elif item < 100:
            value = "00" + str(item) +  secret   
        elif item < 1000:
            value = "0" + str(item) +  secret   
        else:
            value = str(item) + secret

        encoded_value = str.encode(value)
        hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()

        lst_10000_hash.append(str(hash_obj_sha256))
        lst_10000_hash.sort()
    return lst_10000_hash

lst_10_hash = fun_lst_10_hash()
lst_100_hash = fun_lst_100_hash()
lst_1000_hash = fun_lst_1000_hash()
lst_10000_hash = fun_lst_10000_hash()

StatementMeta(testhmacspark, 23, 78, Finished, Available)

In [67]:
def mask_func(col_val):
    col_val_pfe_raw = ""
    dummy_val_string_array = ""
    for i, val in enumerate(col_val):
        if val.isnumeric():
            dummy_val_string_array += str(val)

        if len(dummy_val_string_array) == 4:
            value = dummy_val_string_array + secret
            encoded_value = str.encode(value)
            hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()
            index_pfe = lst_10000_hash.index(hash_obj_sha256)
            if index_pfe < 10:
                value_pfe = "000" + str(index_pfe)
            elif index_pfe < 100:
                value_pfe = "00" + str(index_pfe)
            elif index_pfe < 1000:
                value_pfe = "0" + str(index_pfe)           
            else:
                value_pfe = str(index_pfe)   
            
            col_val_pfe_raw += value_pfe
            dummy_val_string_array = ""
        elif i == len(col_val) - 1:
            if len(dummy_val_string_array) == 3:
                value = dummy_val_string_array + secret
                encoded_value = str.encode(value)
                hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()
                index_pfe = lst_1000_hash.index(hash_obj_sha256)
                if index_pfe < 10:
                    value_pfe = "00" + str(index_pfe)
                elif index_pfe < 100:
                    value_pfe = "0" + str(index_pfe)
                else:
                    value_pfe = str(index_pfe)   
                col_val_pfe_raw += value_pfe
                dummy_val_string_array = ""
            elif len(dummy_val_string_array) == 2:
                value = dummy_val_string_array + secret
                encoded_value = str.encode(value)
                hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()
                index_pfe = lst_100_hash.index(hash_obj_sha256)
                if index_pfe < 10:
                    value_pfe = "0" + str(index_pfe)
                else:
                    value_pfe = str(index_pfe)   
                col_val_pfe_raw += value_pfe
                dummy_val_string_array = ""
            elif len(dummy_val_string_array) == 1:
                value = dummy_val_string_array + secret
                encoded_value = str.encode(value)
                hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()
                index_pfe = lst_10_hash.index(hash_obj_sha256)
                value_pfe = str(index_pfe)   
                col_val_pfe_raw += value_pfe
                dummy_val_string_array = ""

    col_val_pfe = ""
    non_numeric = 0
    for j, val in enumerate(col_val):
        if not(val.isnumeric()):
            col_val_pfe += col_val[j]
            non_numeric += 1
        else:
            col_val_pfe += col_val_pfe_raw[j-non_numeric]
    
    return col_val_pfe


phone = "111-222-333"

phone_pfe = mask_func(phone)
print(phone_pfe)

StatementMeta(testhmacspark, 23, 67, Finished, Available)

927-169-506


In [79]:
# https://tunetotech.com/Post.aspx?post=9
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
#
mask_func_udf = udf(mask_func, StringType())
df2=df.withColumn('Phone', mask_func_udf(df.Phone))

display(df2.limit(10))

StatementMeta(testhmacspark, 23, 79, Finished, Available)

SynapseWidget(Synapse.DataFrame, e3c2a727-92d2-4942-8a9f-57205e861405)

In [80]:
df2.write.mode("overwrite").option("header","true").csv('abfss://gold@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer_masked.txt')

StatementMeta(testhmacspark, 23, 80, Finished, Available)